## 导出特征向量

In [2]:
import os
import h5py

from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import ImageDataGenerator

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [3]:
# 获得当前目录
current_dir = os.getcwd()
# 定义导出特征方法
def export_feature(MODEL, image_size, batch_size, preprocess_input=None):
    # 定义输入的tensor
    input_tensor = Input((image_size[0], image_size[1], 3))
    x = input_tensor
    # 预处理输入
    if preprocess_input:
        x = Lambda(preprocess_input)(x)
    
    # 定义基本模型，以imagenet为权重，不保留顶层的3个全连接网络
    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    
    # 添加全局平均池化层
    model = base_model.output
    model = GlobalAveragePooling2D()(model)
    model = Model(base_model.input, outputs=model)
    
    # 构建图像预处理
    gen = ImageDataGenerator()
    # 创建训练集，指定image_size，不混淆数据，指定一批数据的大小
    train_generator = gen.flow_from_directory(current_dir + '/data/train', image_size, shuffle=False, 
                                              batch_size=batch_size)
    # 创建测试集，指定image_size，不混淆数据，指定一批数据的大小，不返回标签
    test_generator = gen.flow_from_directory(current_dir + "/data/test", image_size, shuffle=False, 
                                             batch_size=batch_size, class_mode=None)
    # 预测生成器，从数据集中获取样本
    train_feature = model.predict_generator(train_generator, train_generator.samples, verbose=1)
    test_feature = model.predict_generator(test_generator, test_generator.samples, verbose=1)
    # 保存对应名称训练后的模型
    with h5py.File("feature_%s.h5"%base_model.name) as h:
        h.create_dataset("train", data=train_feature)
        h.create_dataset("test", data=test_feature)
        h.create_dataset("label", data=train_generator.classes)

In [3]:
export_feature(ResNet50, (224, 224), batch_size=1)

Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
12500/12500 [==============================] - 351s 28ms/step


In [4]:
export_feature(InceptionV3, (299, 299), batch_size=1, preprocess_input=inception_v3.preprocess_input)

87916544/87910968 [==============================] - 1s 0us/step
Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
12500/12500 [==============================] - 522s 42ms/step


In [6]:
export_feature(Xception, (299, 299), batch_size=1, preprocess_input=xception.preprocess_input)

83689472/83683744 [==============================] - 1s 0us/step
Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
12500/12500 [==============================] - 469s 38ms/step


In [4]:
export_feature(DenseNet201, (224, 224), batch_size=1, preprocess_input=densenet.preprocess_input)

76398592/76391848 [==============================] - 1s 0us/step - ETA: 
Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
12500/12500 [==============================] - 986s 79ms/step


In [ ]:
export_feature(DenseNet169, (224, 224), batch_size=1, preprocess_input=densenet.preprocess_input)